<img src='https://www.icos-cp.eu/sites/default/files/2017-11/ICOS_CP_logo.png' width=400 align=right>

# ICOS Carbon Portal Python Libraries

This example uses the library called `icoscp_stilt` which can be used to access results from the [STILT footprint tool](https://www.icos-cp.eu/data-services/tools/stilt-footprint).

Here you can find more [documentation on the libraries](https://icos-carbon-portal.github.io/pylib/) and the source code is available on [GitHub](https://github.com/ICOS-Carbon-Portal/pylib).

For installation of the library locally, more information can be found on [PyPI.org](https://pypi.org/project/icoscp_stilt/). Note that for running the examples locally, authentication is required (see the how_to_authenticate.ipynb notebook).

# Example: Find STILT "stations"


## Import libraries

In [ ]:
from icoscp_stilt import stilt
from icoscp_stilt.stilt import StiltStation #for type annotations
import pandas as pd

## Help

In [ ]:
help(StiltStation)

In [ ]:
help(stilt)

## Find stations:

In [ ]:
all_stations: list[StiltStation] = stilt.list_stations()
stations_by_id: dict[str, StiltStation] = {s.id: s for s in all_stations}

### Lookup by station id

In [ ]:
htm150: StiltStation = stations_by_id['HTM150']
htm150

### Filter by ISO country code

In [ ]:
polish_stations: list[StiltStation] = [s for s in all_stations if s.countryCode == 'PL']
pd.DataFrame(polish_stations)

### Filter by altitude

In [ ]:
high_alts = [s for s in all_stations if s.alt > 300]
pd.DataFrame(high_alts).sort_values(by='alt')

### Filter by simulation time year

In [ ]:
with_2006 = [s for s in all_stations if 2006 in s.years]
pd.DataFrame(with_2006)

### Filter by observation data availability

In [ ]:
with_obs = [s for s in all_stations if s.has_observation_data]
pd.DataFrame(with_obs)

### Filter "proper" ICOS stations

In [ ]:
icos_stations = [s for s in all_stations if s.is_icos_proper]
with pd.option_context('display.max_columns', None, 'display.max_rows', None):
    display(pd.DataFrame(icos_stations))

### Look up available simulation months

In [ ]:
stilt.available_months('KRE250', 2022)

In [ ]:
stilt.available_year_months(htm150)

---
# More examples 
The remainder of this notebook shows examples using the functionalities as they were implemented in the original icoscp_stilt library. These examples still work as before but can be less performant. 

While you are encouraged to switch to using the new functionality whenever possible, some features have so far not been updated. 


## Imports for these examples

In [ ]:
import os
import matplotlib.pyplot as plt
from matplotlib import colors
import datetime as dt
from icoscp_stilt import stiltstation

### Help

In [ ]:
help(stiltstation)

## Output formats available in the original version
### dict (default)

In [ ]:
stiltstation.find(id='HUN115', outfmt='dict')

### pandas

In [ ]:
stiltstation.find(country='HU', outfmt='pandas')

### map (folium, leaflet)
Display the map directly if you are working in a Jupyter Notebook, 
<br> or save the map to html file if you work in python.

In [ ]:
stiltstation.find(country='IT' ,outfmt='map')

#### save the map

In [ ]:
# define the output folder
output_directory = os.path.join(os.path.expanduser('~'), 'output/')
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# this is the actual command to save the map
stiltstation.find(country='FI' ,outfmt='map').save(output_directory + 'myMap.html')

### Availability of data
The flag `outfmt = 'avail'` returns a pandas dataframe with information about the availability of data over time.

In [ ]:
df = stiltstation.find(outfmt = 'avail')

# We can save into a csv-file:
output_directory = os.path.join(os.path.expanduser('~'), 'output/')
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
df.to_csv(output_directory + 'Stilt_availability_' + dt.datetime.now().strftime('%Y%m%d') + '.csv')

df

#### Better view of the availability table
To get better overview we color the values using pandas. <br>
For further reading see: https://pandas.pydata.org/pandas-docs/version/1.1/user_guide/style.html

In [ ]:
def background_gradient(colormap = 'Blues', m =-5, M=17):
    # This function will just run once to produce a list of 13 colors   
    #     - colormap 'Blues' is a gradient color in blue
    #       m = -5 means very light blue, M = 17 means dark blue

    norm = colors.Normalize(m,M)
    normed = norm(range(13))      
    month_hexcolors = [colors.rgb2hex(x) for x in plt.cm.get_cmap(colormap)(normed)]
    return month_hexcolors

def cell_style(column, palette):
    return ['text-align:center; background-color: %s' % palette[val] for val in column]
    
color_palette = background_gradient()
    
# Coloring cells and center the values of the dataframe 
# (The return df_styled is actually a 'Styler' (pandas.io.formats.style.Styler)
year_ls = [y for y in df.columns if isinstance(y,int)]
df_styled = df.style.apply(cell_style, subset = year_ls, palette = color_palette) 

# Also, we might like a title 
df_styled.set_caption('<center style="font-size:16px"> <b>STILT availability</b></center>')

## Search and filter options as implemented in the original version 

In [ ]:
# create a simple function to print the Station name
def stprint(stations):
    if not 'empty' in stations.keys():
        for s in sorted(stations.keys()):
            print(stations[s]['id'],stations[s]['name'], stations[s]['years'])
    else:
        print(stations)

### arbitrary text search

In [ ]:
myStations = stiltstation.find(search='KIT')
stprint(myStations)

#### spatial filters

##### bounding box
approximately scandinavia

In [ ]:
myStations = stiltstation.find(bbox=[(70,5),(55,32)]) 
stprint(myStations)

#### pinpoint
approximately Lund with 200 km distance.

In [ ]:
myStations = stiltstation.find(pinpoint=[55.7,13.1,200])
stprint(myStations)

### temporal
Please be aware, that the actual data is not filtered, the result will return stations where data is **available** according to your filter. Three keywords are available, **sdate**: >= start date, **edate**: <= end date, **>dates**: [ ], a list of dates, NOT a range. If you provide sdate AND edate, stations are returned where data is available within that range.
<br> The following input formats are accepted:
- string: yyyy yyyy-mm yyyy/mm yyyy-mm-dd yyyy/mm/dd 
- float or integer representing a unixtimestamp for seconds since 1970.01.01
- python datetime object
- pandas datetime object

#### start date

In [ ]:
myStations = stiltstation.find(sdate='2020/05')
stprint(myStations)

#### end date

In [ ]:
import datetime
myStations = stiltstation.find(edate='2006-12-31')
stprint(myStations)

In [ ]:
# Using a UNIX Timestamp (seconds sincs 1970-01-01)
import time
now = time.time()
print(now)
myStations = stiltstation.find(edate=now)
len(myStations)

#### range

In [ ]:
myStations = stiltstation.find(sdate= '2018-05-01', edate='2018-06-01')
for s in myStations:
    print(myStations[s]['id'], sorted(myStations[s]['years']))

In [ ]:
myStations = stiltstation.find(sdate= '2018', edate='2018')
stprint(myStations)

#### project
The choice `project = 'icos'` will only return STILT-stations that are ICOS-stations.

In [ ]:
myStations = stiltstation.find(project = 'icos')
stprint(myStations)

### combine filters
Filters are applied in sequence of input and each result is the base for the next filter.
<br>For example you could search for stations in a bounding box and filter the result for a specific country.

In [ ]:
myStations = stiltstation.find(bbox=[(70,5),(55,32)], country='NO') 
stprint(myStations)

In [ ]:
myStations = stiltstation.find(sdate= '2018-05-01', edate='2018-06-01', country='IT')
for s in myStations:
    print(myStations[s]['id'], sorted(myStations[s]['years']))